Nama : Moch Reki Hadiyanto <br>
NIM : 211351083 <br>
Kelas : IF Pagi B <br>

#Import Dataset

Seperti biasa langkah pertama adalah memasukkan datasets yang sudah kita pilih.

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mochrekihadiyanto","key":"b16c456536eb40b8ba5762192c99a882"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [3]:
!kaggle datasets download -d matthieugimbert/french-bakery-daily-sales

  0% 0.00/1.94M [00:00<?, ?B/s]
100% 1.94M/1.94M [00:00<00:00, 33.7MB/s]


In [4]:
!unzip french-bakery-daily-sales.zip -d dataset
!ls dataset

Archive:  french-bakery-daily-sales.zip
  inflating: dataset/Bakery sales.csv  
'Bakery sales.csv'


#Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import association_rules, apriori
import plotly.express as px
import re
import pickle

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



#Data Discovery

Kita memasukkan file csv yang tadi telah diextract pada sebuah variable yang bernama df.

In [ ]:
df = pd.read_csv('dataset/Bakery sales.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Melihat 5 data awal dari datasets.

In [ ]:
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,date,time,ticket_number,article,Quantity,unit_price
0,0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,"0,90 €"
1,1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €"
2,4,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €"
3,5,2021-01-02,09:14,150041.0,PAIN,1.0,"1,15 €"
4,8,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €"


Kita bisa lihat dibawah mayoritas datatype dari datasetsnya adalah object.

In [ ]:
df.info()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234005 entries, 0 to 234004
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     234005 non-null  int64  
 1   date           234005 non-null  object 
 2   time           234005 non-null  object 
 3   ticket_number  234005 non-null  float64
 4   article        234005 non-null  object 
 5   Quantity       234005 non-null  float64
 6   unit_price     234005 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 12.5+ MB


In [ ]:
df.describe()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,ticket_number,Quantity
count,234005.000000,234005.000000,234005.000000
mean,255205.035854,219201.258738,1.538377
std,147501.625992,40053.223896,1.289603
min,0.000000,150040.000000,-200.000000
25%,127979.000000,184754.000000,1.000000
50%,254573.000000,218807.000000,1.000000
75%,382911.000000,253927.000000,2.000000
max,511395.000000,288913.000000,200.000000


Diatas menunjakkan nilai mean, min hingga max dari kolom-kolom yang numeric.

In [ ]:
df.isnull().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Unnamed: 0       0
date             0
time             0
ticket_number    0
article          0
Quantity         0
unit_price       0
dtype: int64

In [ ]:
df['article'].sort_values().unique()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



array(['.', '12 MACARON', 'ARMORICAIN', 'ARTICLE 295', 'BAGUETTE',
       'BAGUETTE APERO', 'BAGUETTE GRAINE', 'BANETTE', 'BANETTINE',
       'BOISSON 33CL', 'BOTTEREAU', 'BOULE 200G', 'BOULE 400G',
       'BOULE POLKA', 'BRIOCHE', 'BRIOCHE DE NOEL', 'BRIOCHETTE',
       'BROWNIES', 'BUCHE 4PERS', 'BUCHE 6PERS', 'BUCHE 8PERS',
       'CAFE OU EAU', 'CAKE', 'CAMPAGNE', 'CARAMEL NOIX',
       'CEREAL BAGUETTE', 'CHAUSSON AUX POMMES', 'CHOCOLAT',
       'CHOU CHANTILLY', 'COMPLET', 'COOKIE', 'COUPE', 'CROISSANT',
       'CROISSANT AMANDES', 'CRUMBLE', 'CRUMBLECARAMEL OU PISTAE',
       'DELICETROPICAL', 'DEMI BAGUETTE', 'DEMI PAIN', 'DIVERS BOISSONS',
       'DIVERS BOULANGERIE', 'DIVERS CONFISERIE', 'DIVERS PATISSERIE',
       'DIVERS SANDWICHS', 'DIVERS VIENNOISERIE', 'DOUCEUR D HIVER',
       'ECLAIR', 'ECLAIR FRAISE PISTACHE', 'ENTREMETS', 'FICELLE',
       'FINANCIER', 'FINANCIER X5', 'FLAN', 'FLAN ABRICOT',
       'FONDANT CHOCOLAT', 'FORMULE PATE', 'FORMULE PLAT PREPARE',
       'F

Wow, diatas merupakan nilai unique dari kolom article, yang artinya itu merupakan menu yang tersedia pada toko french ini. Cukup banyak ya.

In [ ]:
df.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(234005, 7)

Nah terakhir, kita bisa lihat bahwa datasets ini memiliki 234,005 jumlah baris data dengan 7 kolom.

#EDA

Pada proses EDA ini, saya harus melakukan banyak modifikasi data jadi dibagian ini juga bisa dibilang bagian dari pre-processing.

In [ ]:
df["date"] = pd.to_datetime(df["date"] + " " + df["time"])
df.rename(columns={"date":"date_time"},inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Code diatas digunakan untuk menyatukan kolom data & time lalu mengubah namanya menjadi kolom date_time.

Sedangkan code dibawah itu untuk memisahkan nilai year, month, day, dan dayofweek pada kolomnya sendiri.

In [ ]:
df["Y"] = df["date_time"].dt.year
df["M"] = df["date_time"].dt.month
df["D"] = df["date_time"].dt.day
df["WD"] = df["date_time"].dt.dayofweek + 1
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,date_time,time,ticket_number,article,Quantity,unit_price,Y,M,D,WD
0,0,2021-01-02 08:38:00,08:38,150040.0,BAGUETTE,1.0,"0,90 €",2021,1,2,6
1,1,2021-01-02 08:38:00,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €",2021,1,2,6
2,4,2021-01-02 09:14:00,09:14,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €",2021,1,2,6
3,5,2021-01-02 09:14:00,09:14,150041.0,PAIN,1.0,"1,15 €",2021,1,2,6
4,8,2021-01-02 09:25:00,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €",2021,1,2,6
...,...,...,...,...,...,...,...,...,...,...,...
234000,511387,2022-09-30 18:52:00,18:52,288911.0,COUPE,1.0,"0,15 €",2022,9,30,5
234001,511388,2022-09-30 18:52:00,18:52,288911.0,BOULE 200G,1.0,"1,20 €",2022,9,30,5
234002,511389,2022-09-30 18:52:00,18:52,288911.0,COUPE,2.0,"0,15 €",2022,9,30,5
234003,511392,2022-09-30 18:55:00,18:55,288912.0,TRADITIONAL BAGUETTE,1.0,"1,30 €",2022,9,30,5


Selanjutnya adalah mengubah kolom Quantity dengan datatype integer. Dan menghilangkan simbol euro serta mengubah ',' menjadi '.' lalu mengubah datatypenya menjadi float agar mudah untuk diproses nantinya.

In [ ]:
df.Quantity = df.Quantity.astype(int)
df.unit_price = df.unit_price.str.replace(' €', '').str.replace(',','.').astype(float)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Kita bisa mendapatkan nilai Revenue(keuntungan) dengan mengkalikan kolom Quantity dengan unit_price.

In [ ]:
df["REV"] = df.Quantity * df.unit_price

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Kita akan melakukan groupby perbulan untuk melihat volume penjualan, jumlah client, dan revenue yang dihasilkan dibulan itu.

In [ ]:
monthly = df.groupby("M", as_index=False).agg(total_sales_volume=("Quantity","sum"), client_num=("ticket_number","count"),total_rev=("REV","sum"))
monthly.loc[:8,monthly.columns.difference(["M"])] = monthly.loc[:8,monthly.columns.difference(["M"])].applymap(lambda x:x/2)
monthly

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,M,total_sales_volume,client_num,total_rev
0,1,9214.0,6417.5,15472.185
1,2,11682.5,7806.0,17227.325
2,3,12791.5,8743.0,19717.105
3,4,15739.5,10509.5,24660.850
4,5,18556.0,12043.0,29451.675
5,6,16014.0,10767.0,24948.275
6,7,28764.5,17917.5,44398.835
7,8,33436.5,20239.5,51269.000
8,9,14354.5,9712.5,22275.650
9,10,14971.0,9861.0,22704.350


In [ ]:
fig = px.line(monthly,x="M",y=["client_num","total_sales_volume","total_rev"],title="Monthly number of clients, total sales volume and total revenue")
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Nah, diatas menunjukkan ketiga data yang tadi telah dikelompokkan berdasarkan bulan. Kita bisa lihat bahwa semuanya mengelonjak tinggi dari bulan 6 dengan bulan ke 8 sebagai puncaknya. Itu merupakan bulan penting bagi toko.

Selanjutnya kita akan melihat jumlah revenue yang dihasilkan masing-masing bulan dalam setahun. (based on percentage tentunya)

In [ ]:
fig = px.pie(monthly,names="M",values="total_rev",hole=0.5,title="How much revenue does each month account for?")
fig.update_traces(textposition='outside',textinfo='percent+label',sort=False)
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Nah, betul saja bulan ke 8 menghasilkan 16.6% revenue dari tahun ini, diikuti dengan bulan ke 7 dan dilanjut dengan bulan ke 5.

Selanjutnya kita akan melihat based on day(daily), sama seperti diatas, kita akan melihat volume Quantity, jumlah client dan total revenue yang dihasilkan.

In [ ]:
daily = df.groupby("D", as_index=False).agg(total_sales_volume=("Quantity","sum"), client_num=("ticket_number","count"),total_rev=("REV","sum"))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
fig = px.line(daily,x="D",y=["client_num","total_sales_volume","total_rev"],title="Daily number of clients, total sales volume and total revenue")
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Penjualan meningkat pada pertengahan bulan yaitu tanggal 13 hingga 15 dengan puncak sekitar tanggal 14.

Kita akan lihat dalam seminggu itu dihari keberapakah penjualan meningkat secara pesat.

In [ ]:
weekly = df.groupby("WD", as_index=False).agg(total_sales_volume=("Quantity","sum"), client_num=("ticket_number","count"),total_rev=("REV","sum"))
fig = px.line(weekly,x="WD",y=["client_num","total_sales_volume","total_rev"],title="Number of clients, total sales volume and total revenue inside a week")
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Ya!, saya tidak terkejut karena weekend memanglah hari yang tepat untuk menikmati makanan manis, maka dari itu dihari-hari weekendlah penjualan selalu meningkat.

Kita akan melihat dengan lebih merinci lagi dengan melihat tiap jamnya dalam sehari, kita akan melihat hal yang sama yaitu jumlah client, volume quantity penjualan dan total revenue.

In [ ]:
df["hr"] = df["date_time"].dt.hour
hourly = df.groupby("hr", as_index=False).agg(total_sales_volume=("Quantity","sum"), client_num=("ticket_number","count"),total_rev=("REV","sum"))
fig = px.line(hourly,x="hr",y=["client_num","total_sales_volume","total_rev"],title="number of clients, total sales volume and total revenue inside a day")
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Hasil yang tidak mengejutkan juga, penjualan meningkat pada jam-jam istirahat.

#Preprocessing

Kita akan melanjutkan tahap Preprocessing di tahap yang sebenarnya.

In [ ]:
transactions = df['ticket_number'].nunique()

print('transactions in the dataset: ' + str(transactions))

transactions in the dataset: 136451


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Kita memasukkan ticket_number(number penjualan) pada variable transaction dan terdapat 136451 transaksi pada datasets.

In [ ]:
df.loc[(df['article'] == 'REDUCTION SUCREES 24') | (df['article'] == 'REDUCTION SUCREES 12')]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,Unnamed: 0,date_time,time,ticket_number,article,Quantity,unit_price,Y,M,D,WD,REV,hr
130982,285022,2021-12-31 09:56:00,09:56,227099.0,REDUCTION SUCREES 12,1,10.0,2021,12,31,5,10.0,9
130998,285046,2021-12-31 10:04:00,10:04,227103.0,REDUCTION SUCREES 12,1,10.0,2021,12,31,5,10.0,10
131040,285128,2021-12-31 10:34:00,10:34,227123.0,REDUCTION SUCREES 12,1,10.0,2021,12,31,5,10.0,10
131045,285135,2021-12-31 10:35:00,10:35,227124.0,REDUCTION SUCREES 12,1,10.0,2021,12,31,5,10.0,10
131219,285455,2021-12-31 11:57:00,11:57,227197.0,REDUCTION SUCREES 12,1,10.0,2021,12,31,5,10.0,11
131303,285605,2021-12-31 12:43:00,12:43,227230.0,REDUCTION SUCREES 12,1,10.0,2021,12,31,5,10.0,12
131324,285652,2021-12-31 16:09:00,16:09,227243.0,REDUCTION SUCREES 12,1,10.0,2021,12,31,5,10.0,16
131348,285690,2021-12-31 16:29:00,16:29,227250.0,REDUCTION SUCREES 24,1,15.0,2021,12,31,5,15.0,16
177284,386852,2022-06-05 09:31:00,09:31,255007.0,REDUCTION SUCREES 12,1,10.0,2022,6,5,7,10.0,9


In [ ]:
df['article_cat'] = df['article'].apply(lambda x: re.sub('[0-9](.*)','',x))\
.str.strip()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
df['article_cat'].nunique()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



131

In [ ]:
df['article_cat'].sort_values().unique()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



array(['', '.', 'ARMORICAIN', 'ARTICLE', 'BAGUETTE', 'BAGUETTE APERO',
       'BAGUETTE GRAINE', 'BANETTE', 'BANETTINE', 'BOISSON', 'BOTTEREAU',
       'BOULE', 'BOULE POLKA', 'BRIOCHE', 'BRIOCHE DE NOEL', 'BRIOCHETTE',
       'BROWNIES', 'BUCHE', 'CAFE OU EAU', 'CAKE', 'CAMPAGNE',
       'CARAMEL NOIX', 'CEREAL BAGUETTE', 'CHAUSSON AUX POMMES',
       'CHOCOLAT', 'CHOU CHANTILLY', 'COMPLET', 'COOKIE', 'COUPE',
       'CROISSANT', 'CROISSANT AMANDES', 'CRUMBLE',
       'CRUMBLECARAMEL OU PISTAE', 'DELICETROPICAL', 'DEMI BAGUETTE',
       'DEMI PAIN', 'DIVERS BOISSONS', 'DIVERS BOULANGERIE',
       'DIVERS CONFISERIE', 'DIVERS PATISSERIE', 'DIVERS SANDWICHS',
       'DIVERS VIENNOISERIE', 'DOUCEUR D HIVER', 'ECLAIR',
       'ECLAIR FRAISE PISTACHE', 'ENTREMETS', 'FICELLE', 'FINANCIER',
       'FINANCIER X', 'FLAN', 'FLAN ABRICOT', 'FONDANT CHOCOLAT',
       'FORMULE PATE', 'FORMULE PLAT PREPARE', 'FORMULE SANDWICH',
       'FRAISIER', 'FRAMBOISIER', 'GACHE', 'GAL FRANGIPANE',
       'GA

Selanjutnya kita akan menghilangkan beberapa nilai yang terdapat pada kolom article, lalu memasukkannya pada kolom article_cat.

In [ ]:
delete_list = ['', ' ', '.', 'ARTICLE', 'THE']

df = df[df['article_cat'].isin(delete_list) == False]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
df['article_cat'].nunique()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



127

Sekarang terdapat 127 nilai unique pada kolom article_cat(menu) setelah dihilangkannya nilai '', ' ', '.', 'ARTICLE', dan 'THE'.

In [ ]:
transactions = df['ticket_number'].nunique()

print('transactions in the dataset: ' + str(transactions))

transactions in the dataset: 136438


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Dan jumlah transaksinya pun berkurang menjadi 136538 transaksi.

In [ ]:
products = df['article_cat'].nunique()

print('products in the dataset: ' + str(products))

products in the dataset: 127


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
sales_by_ticket = pd.DataFrame(df.groupby(['ticket_number', 'article_cat'])['Quantity'].count()).reset_index()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Membuat variable untuk menampung jumlah penjualan berdasarkan number ticket, jadi jika number ticketsnya sama maka itu termasuk dalam satu sales.

In [ ]:
sales_grouped = pd.crosstab(index=sales_by_ticket.ticket_number, columns=sales_by_ticket.article_cat)\
.rename_axis(None)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
sales_grouped = sales_grouped.applymap(lambda x: True if x>= 1 else False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
sales_grouped.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



article_cat,ARMORICAIN,BAGUETTE,BAGUETTE APERO,BAGUETTE GRAINE,BANETTE,BANETTINE,BOISSON,BOTTEREAU,BOULE,BOULE POLKA,...,TARTELETTE FRAISE,TRADITIONAL BAGUETTE,TRAITEUR,TRIANGLES,TROIS CHOCOLAT,TROPEZIENNE,TROPEZIENNE FRAMBOISE,TULIPE,VIENNOISE,VIK BREAD
150040.0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150041.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150042.0,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
150043.0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150044.0,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Kita bisa lihat sekarang tiap tiap menunya memiliki nilai true (jika ada pada pembelian) dan nilai false (jika tidak ada).

#Modeling

Kita akan lanjut dengan tahap modeling, disini saya akan menggunakan metric support untuk membuat modelnya karena data yang tersedia dan yang akan dihasilkan akan sangat banyak sekali.

In [ ]:
frequent_itemsets = apriori(sales_grouped, min_support=0.005,use_colnames=True, max_len=4)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
rules = association_rules(frequent_itemsets, metric="support",min_threshold=0.005)

rules.head(15)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(BANETTE),(BAGUETTE),0.110724,0.111941,0.006552,0.059178,0.528652,-0.005842,0.943918,-0.500653
1,(BAGUETTE),(BANETTE),0.111941,0.110724,0.006552,0.058535,0.528652,-0.005842,0.944566,-0.500995
2,(BAGUETTE),(BOULE),0.111941,0.049070,0.005131,0.045833,0.934025,-0.000362,0.996607,-0.073679
3,(BOULE),(BAGUETTE),0.049070,0.111941,0.005131,0.104556,0.934025,-0.000362,0.991752,-0.069144
4,(COUPE),(BAGUETTE),0.142365,0.111941,0.011273,0.079180,0.707341,-0.004664,0.964422,-0.325430
5,(BAGUETTE),(COUPE),0.111941,0.142365,0.011273,0.100701,0.707341,-0.004664,0.953670,-0.317825
6,(CROISSANT),(BAGUETTE),0.083892,0.111941,0.010789,0.128604,1.148855,0.001398,1.019122,0.141433
7,(BAGUETTE),(CROISSANT),0.111941,0.083892,0.010789,0.096379,1.148855,0.001398,1.013820,0.145900
8,(PAIN AU CHOCOLAT),(BAGUETTE),0.077171,0.111941,0.008781,0.113781,1.016438,0.000142,1.002076,0.017524
9,(BAGUETTE),(PAIN AU CHOCOLAT),0.111941,0.077171,0.008781,0.078439,1.016438,0.000142,1.001376,0.018210


#Visualisasi hasil algorithm.

Dibawah menunjukkan bahwa COUPE merupakan produk yang paling sering dibeli dengan 83% confidence akan selalu memiliki pasangan pembelian, seperti COUPE dan BOULE sangat sering muncul pada pembelian.

In [ ]:
rules.sort_values(['confidence','lift','support'],ascending=[False, False, True]).head(15)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
86,"(TRADITIONAL BAGUETTE, BOULE)",(COUPE),0.011185,0.142365,0.009382,0.838794,5.891856,0.007789,5.320126,0.839665
23,(BOULE),(COUPE),0.049070,0.142365,0.040788,0.831217,5.838634,0.033802,5.081297,0.871491
45,(SEIGLE),(COUPE),0.009711,0.142365,0.007820,0.805283,5.656466,0.006438,4.404521,0.831284
28,(CAMPAGNE),(COUPE),0.028526,0.142365,0.022809,0.799589,5.616470,0.018748,4.279379,0.846088
91,"(CAMPAGNE, TRADITIONAL BAGUETTE)",(COUPE),0.006853,0.142365,0.005453,0.795722,5.589307,0.004477,4.198370,0.826753
33,(COMPLET),(COUPE),0.022963,0.142365,0.016960,0.738589,5.187996,0.013691,3.280794,0.826220
51,(VIK BREAD),(COUPE),0.022912,0.142365,0.016813,0.733845,5.154673,0.013552,3.222316,0.824901
98,"(TRADITIONAL BAGUETTE, COMPLET)",(COUPE),0.008385,0.142365,0.006142,0.732517,5.145347,0.004948,3.206322,0.812462
116,"(TRADITIONAL BAGUETTE, VIK BREAD)",(COUPE),0.011720,0.142365,0.008568,0.731082,5.135263,0.006900,3.189205,0.814817
41,(MOISSON),(COUPE),0.022684,0.142365,0.016183,0.713409,5.011123,0.012954,2.992537,0.819023


#Export dengan Pickle